<a href="https://colab.research.google.com/github/starkdg/pyConvnetPhash/blob/master/train_pcanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow_hub as hub

model_dir = "/gdrive/My Drive/models"
module_inception_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1"
module_mobilenetv2_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/2"

module = hub.Module(module_mobilenetv2_url)
target_height, target_width = hub.get_expected_image_size(module)

n_inputs = 1792
n_hidden = 256

# location of testing .tfrecord files
training_files_dir = "/gdrive/My Drive/imageset/train"
validation_files_dir = "/gdrive/My Drive/imageset/validation"
testing_files_dir = "/gdrive/My Drive/imageset/test"

batch_size = 10
epochs = 15
steps = 2000
learning_rate = 0.0001
norm_constant = 5.0                # input normalization constant (divide input features by)
transfer_function = tf.identity    # activation on hidden layer - other values: tf.identity, tf.nn.sigmoid, tf.nn.relu
model_tag = "linear"               # location and file of exported frozen model file: "linear", "sigmoid", "relue"

frozen_model = "/gdrive/My Drive/models/pca_autoencoder/mobilenetv2_pca_autoenc_{0}to{1}_frozen_model-{2}.pb".format(n_inputs, n_hidden, model_tag)

In [0]:
weights = dict()
with tf.variable_scope("weights", reuse=tf.AUTO_REUSE):
  weights['w1'] = tf.get_variable('w1', shape=[n_inputs, n_hidden], trainable=True, initializer=tf.contrib.layers.xavier_initializer())
  weights['b1'] = tf.Variable(tf.zeros([n_hidden], dtype=tf.float32), trainable=True)
                                  
  weights['w2'] = tf.transpose(weights['w1'])
  weights['b2'] = tf.Variable(tf.zeros([n_inputs], dtype=tf.float32), trainable=True)

In [0]:
def get_tfrecord_files(path):
  files = []
  for entry in os.scandir(path):
    if entry.is_file() and entry.name.endswith('.tfrecord'):
             files.append(entry.path)
  return files
  
  
def _parse_example(example):
  features = {'height': tf.FixedLenFeature([], tf.int64),
              'width': tf.FixedLenFeature([], tf.int64),
              'image_raw': tf.FixedLenFeature([], tf.string)}
  parsed_features = tf.parse_single_example(example, features)
  img = tf.io.decode_raw(parsed_features['image_raw'], tf.uint8)
  height = tf.cast(parsed_features['height'], tf.int32)
  width = tf.cast(parsed_features['width'], tf.int32)

  img_reshaped = tf.manip.reshape(img, [height, width, 3])
  imgfl = tf.image.convert_image_dtype(img_reshaped, dtype=tf.float32)
  img_norm = tf.expand_dims(imgfl, 0)
  img_resized = tf.image.resize_bicubic(img_norm, [target_height, target_width])
  img_resized = tf.squeeze(img_resized, 0)
  return img_resized


def input_function(path, batch_size=1, num_epochs=None, shuffle=False):
  tfrecords = get_tfrecord_files(path)
  dataset = tf.data.TFRecordDataset(tfrecords)
  dataset = dataset.map(_parse_example)
  if (shuffle):
    dataset = dataset.shuffle(10000)
  dataset = dataset.batch(batch_size).repeat(num_epochs)
  iterator = dataset.make_initializable_iterator()
  return iterator

In [0]:
x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="input")  

def create_pca_autoencoder(learning_rate, norm_constant):

  norm = tf.constant(norm_constant, tf.float32, name="normalization_constant")
  
  # input normalization
  norm_x = tf.math.xdivy(x, norm)

  # input_dims -> n_hidden
  multadd1 = tf.add(tf.matmul(norm_x, weights['w1']), weights['b1'])
  hidden = transfer_function(multadd1, name="output")
    
  # reconstruction  
  # n_hidden ->  input_dims
  y = tf.identity(tf.add(tf.matmul(hidden, weights['w2']), weights['b2']), name="y")
      
  with tf.variable_scope("opt", reuse=tf.AUTO_REUSE): 
    cost = tf.reduce_mean(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=norm_x, logits=y), axis=1, keepdims=True))
    optimizer = tf.train.AdamOptimizer(learning_rate) 
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      opt = optimizer.minimize(cost)
     
  return hidden, cost, opt


In [0]:
def train_pca_model(training_files_dir,
                        validation_files_dir,
                        testing_files_dir,
                        batch_size, epochs, steps, learning_rate, norm_constant, model_tag):
    period_size = 100
        
    training_iter = input_function(training_files_dir, batch_size)
    training_images = training_iter.get_next()
    training_features = module(training_images)
        
    validation_iter = input_function(validation_files_dir, batch_size)
    validation_images = validation_iter.get_next()
    validation_features = module(validation_images)
        
    testing_iter = input_function(testing_files_dir, 100)
    testing_images = testing_iter.get_next()
    testing_features = module(testing_images)
        
    out1, recon_cost, train_op = create_pca_autoencoder(learning_rate, norm_constant)
   
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
  
    train_losses = []
    valid_losses = []
    print("Train PCA Autoencoder model")
    for i in range(epochs):
      sess.run([training_iter.initializer, validation_iter.initializer])
      iteration = 0
      total_cost = 0.
      total_val_cost = 0.
      while True:
        try:
          Xtrain = sess.run(training_features)
          train_cost, opt = sess.run([recon_cost, train_op], feed_dict={x: Xtrain})
          if (iteration % period_size == 0):
            Xvalid = sess.run(validation_features)
            validation_cost = sess.run(recon_cost, feed_dict={x: Xvalid})
            total_cost += train_cost
            total_val_cost += validation_cost
          iteration = iteration + 1
        except tf.errors.OutOfRangeError:
          break
        if (iteration > steps):
          break
                
      steps_taken = iteration//period_size
      avg_train_loss = total_cost/steps_taken
      avg_val_loss = total_val_cost/steps_taken
      print("epoch {0} training cost {1} valid. cost {2}".format(i+1, avg_train_loss, avg_val_loss))
      train_losses.append(avg_train_loss)
      valid_losses.append(avg_val_loss)
   
                         
    plt.figure(101)
    plt.plot(train_losses)
    plt.plot(valid_losses)
    plt.plot()
    plt.title("PCA Autoencoder {0}->{1}".format(n_inputs, n_hidden))
    plt.xlabel("epochs")
    plt.ylabel("cost")
    plt.legend(["training", "validation"], loc="upper right")
    plt.show()
    
    w1 = weights['w1'].eval(session=sess)
    b1 = weights['b1'].eval(session=sess)
    b2 = weights['b2'].eval(session=sess)
    
    nbins = 64
    plt.figure(102)
    plt.hist((w1.ravel()), bins=nbins, histtype='bar', stacked=True)
    plt.legend(['w1'], loc='upper right')
    plt.title('Histogram of Weights')
    plt.show()
    
    plt.figure(103)
    plt.hist((b1.ravel(), b2.ravel()), bins=nbins, histtype='bar', stacked=True)
    plt.legend(['b1', 'b2'], loc='upper right')
    plt.title('Histogram of Biases')
    plt.show()
    
    print("run test on 100 images")
    sess.run([testing_iter.initializer])
    Xtest = sess.run(testing_features)
    testing_cost = sess.run([recon_cost], feed_dict={x: Xtest})
    print("test cost = {0}".format(testing_cost))
    
    graphdef = tf.get_default_graph().as_graph_def()
    subgraphdef = tf.graph_util.extract_sub_graph(graphdef, ['output'])
    subgraphdef = tf.graph_util.remove_training_nodes(subgraphdef, protected_nodes=['output'])
    subgraphdef_frozen = tf.graph_util.convert_variables_to_constants(sess, subgraphdef, ['output'])
    
    print("write model: ", frozen_model)
    with tf.gfile.GFile(frozen_model, "wb") as f:
      f.write(subgraphdef_frozen.SerializeToString())
        
    sess.close() 

In [0]:
print("Train autoencoder")
print("training files: ", training_files_dir)
print("validation files: ", validation_files_dir)
print("testing files: ", testing_files_dir)
print("learning_rate: ", learning_rate)
print("batch size: ", batch_size)
print("epochs: ", epochs)
print("steps: ", steps)
print("norm constant: ", norm_constant)

train_pca_model(training_files_dir,
                    validation_files_dir,
                    testing_files_dir,
                    batch_size, epochs, steps,
                    learning_rate, norm_constant, model_tag)
